In [ ]:
import os
import sys
sys.path.insert(0, "..")
import gc
gc.enable()

In [ ]:
sys.path.insert(0, os.getcwd())
from models.mask_rcnn import MRCNN
from utils.utilities import *
import tensorflow as tf
import gc
gc.enable()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

mrcnn = MRCNN()

In [ ]:
from utils.DataLoader import  DataLoader
from models.mask_rcnn import DetectorDataset

dl = DataLoader()
train_df, valid_df = dl.train_split(0.1, filterNan=True)

dd_train = DetectorDataset(train_df, shape=(256, 256))
dd_train.prepare()
dd_valid = DetectorDataset(valid_df, shape=(256, 256))
dd_valid.prepare()

print(train_df.shape)
print(valid_df.shape)

In [ ]:
h = mrcnn.train(dd_train, dd_valid, epochs=10, layers='heads')

In [ ]:
mrcnn.show_loss(h)

In [ ]:
#mrcnn = MRCNN('/home/gmandi/Documents/DSIT/Deep-Learning/AIRBUS-Ship_Detection/models/mask-rcnn20200903T2344/mask_rcnn_mask-rcnn_0001.h5')
mrcnn.examine_infer(dd_valid)